In [1]:
!pip list

Package                   Version
------------------------- ---------------
accelerate                1.8.1
aiohappyeyeballs          2.6.1
aiohttp                   3.12.13
aiosignal                 1.4.0
annotated-types           0.7.0
anyio                     4.9.0
appdirs                   1.4.4
argon2-cffi               25.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.5
asyncmy                   0.2.10
attrs                     25.3.0
babel                     2.17.0
beautifulsoup4            4.13.4
bleach                    6.2.0
blobfile                  3.0.0
boto3                     1.39.4
botocore                  1.39.4
certifi                   2025.6.15
cffi                      1.17.1
charset-normalizer        3.4.2
click                     8.2.1
cohere                    5.16.1
colorama                  0.4.6
comm                      0.2.2
contourpy                 1.3.2
cryptography

In [2]:
from typing import TypedDict, List, Annotated
from langchain_core.documents import Document
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import os
from langgraph.graph import StateGraph
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [3]:
# 환경 변수 설정
load_dotenv()
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")  

# FAISS 벡터 DB 불러오기
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large-instruct")
vectorstore = FAISS.load_local("card_QA_faiss_db", embedding_model,allow_dangerous_deserialization=True)


C:\Users\Playdata\AppData\Local\Temp\ipykernel_8788\1982410193.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large-instruct")


In [4]:
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall
import json
import pandas as pd
import matplotlib.pyplot as plt
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.retrievers import BM25Retriever, EnsembleRetriever
import cohere

In [ ]:
# 데이터 로드
with open('faq_for_vectordb.json', 'r', encoding='utf-8') as file:
    faq_data = json.load(file)

# 질문 데이터셋
questions = [item["title"] for item in faq_data]
references = [item["page_content"] for item in faq_data]

# FAISS 벡터 DB 불러오기
embedding_model = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large-instruct")
vectorstore = FAISS.load_local("card_QA_faiss_db", embedding_model,allow_dangerous_deserialization=True)

# 전체 문서 (BM25용)
all_docs = vectorstore.docstore._dict.values()

# BM25 retriever
bm25_retriever = BM25Retriever.from_documents(all_docs)
bm25_retriever.k = 5

# faiss retriever
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# Hybrid (ensemble) retriever
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    weights=[0.5, 0.5]
)

# RetrievalQA 모델 초기화 (유사도 검색을 통한 질의응답)
llm = ChatOpenAI(model_name='gpt-4.1-mini-2025-04-14')
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=ensemble_retriever
)

# 질문을 통해 모델 답변 생성
def get_answer_from_retrieval(question):
    hybrid_docs = ensemble_retriever.invoke(question)
    # Cohere Rerank용 문서 구성
    rerank_docs = [
        {
            "text": doc.page_content.strip(),
            "metadata": doc.metadata,
            "id": doc.metadata.get("source", f"doc_{i}")
        }
        for i, doc in enumerate(hybrid_docs)
        if doc.page_content.strip()
    ]
    rerank_result = co.rerank(
        model="rerank-multilingual-v3.0",
        query=question,
        documents=rerank_docs,
        top_n=3,
        return_documents=True
    )

    reranked_docs = [
        Document(
            page_content=res.document.text,
            metadata={**res.document.metadata, "score": res.relevance_score}
        )
        for res in rerank_result.results
    ]
    prompt = ChatPromptTemplate.from_template(
        """
        문서: {docs}
        질문: {question}
        위 문서들을 참고해서 질문에 답변해줘.
        """
    )
    docs_content = "\n---\n".join([doc.page_content for doc in reranked_docs])
    chain = prompt | ChatOpenAI(model="gpt-4.1-mini-2025-04-14")
    answer = chain.invoke({"docs": docs_content, "question": question}).content
    print("\n[llm_answer_node] 생성된 답변:", answer)
    return answer

generated_answers = [get_answer_from_retrieval(q) for q in questions]

# 평가를 위한 데이터셋 준비
data = {
    "question": questions,
    "answer": generated_answers, # 모델에서 생성한 답변
    "contexts": [[ref] for ref in references], # 원본 문서 내용 (context)
    "reference": references # 정답(reference)
}

dataset = Dataset.from_dict(data)

# RAGAS 평가 실행
result = evaluate(
    dataset=dataset,
    metrics=[faithfulness, answer_relevancy, context_precision, context_recall]
)

# 평가 결과 DataFrame 출력
result_df = result.to_pandas()
print(result_df)

# 평가 결과 시각화
fig, ax = plt.subplots(figsize=(12, 6))
result_df.mean().plot(kind='bar', ax=ax, color=['#4caf50', '#2196f3', '#ff9800', '#e91e63'])
plt.title('RAGAS Evaluation Metrics (LangChain Retrieval)')
plt.ylabel('Score')
plt.xlabel('Metrics')
plt.ylim(0, 1)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--')

for p in ax.patches:
    ax.annotate(f'{p.get_height():.2f}', (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center', xytext=(0, 10), textcoords='offset points')

plt.tight_layout()
plt.show()


C:\Users\Playdata\AppData\Local\Temp\ipykernel_8788\2540348060.py:30: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name='gpt-4.1-mini-2025-04-14')
C:\Users\Playdata\AppData\Local\Temp\ipykernel_8788\2540348060.py:39: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain.run(question)


KeyboardInterrupt: 